In [2]:
! pip install bnlp_toolkit

Keyring is skipped due to an exception: 'keyring.backends'


In [ ]:
!wget https://huggingface.co/sagorsarker/bangla_word2vec/resolve/main/bangla_word2vec_gen4.zip
!unzip bangla_word2vec_gen4.zip
!rm -rf bangla_word2vec_gen4.zip

In [5]:
bwv = BengaliWord2Vec()
model_path = "bangla_word2vec/bnwiki_word2vec.model"


def vectorize_word(word):
    return bwv.generate_word_vector(model_path, word)
vectorize_word("নৃগোষ্ঠী")

NameError: name 'BengaliWord2Vec' is not defined

### Preprocess Dataset

In [ ]:
import string
def preprocess_word(word): ## hANDLE DARI ।
  translate_table = dict((ord(char), None) for char in string.punctuation)   
  word =  word.translate(translate_table)
  return word.replace("।", "")

print(preprocess_word("পর,"))

In [ ]:
def getDatasetFromIBOFile(filename): #nlp_hackathon_bd_2023/data/dev.txt"
    f = open(filename, "r")
    content = f.read()
    
    lines = content.split("\n")
    
    dataset = []
    for line in lines:
        parts = line.split(" _")
        word = parts[0]
        word = preprocess_word(word)
        label = parts[len(parts)-1]


        # Special annoying cases
        if word=="ম্যাককম্ব" or word=='' or word=="ব্রাংম্যান" or word=='ডাম্পসন': continue

        dataset.append([word,label])
    return dataset

In [ ]:
def encodeClass(Y):
    classes = []
    classes = [val for val in Y if val not in classes]
    
    new_y = []
    for cls in Y:
        for idx, val in enumerate(classes):
            if cls==val:
                new_y.append(idx)
                break
    return new_y
                
# encodeClass(Y)

In [ ]:
def preprocess_dataset(dataset):
    X = []
    Y = []
    for row in dataset:
      word = row[0]
      label = row[1]

      print(word)

      X.append(vectorize_word(word))
      Y.append(label)
    
    Y = encodeClass(Y)
    
    Y = np.array(Y)
    X = np.array(X)
    return X, Y
    

In [ ]:
DATA_SIZE = 100
dataset = getDatasetFromIBOFile("nlp_hackathon_bd_2023/data/dev.txt")[:DATA_SIZE]
X, Y = preprocess_dataset(dataset)

### Model!

In [ ]:
import numpy as np
import random

import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [ ]:
model = Sequential([
    Dense(units=16, input_shape=(100, ), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=5, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=X, y=Y, batch_size=10, epochs=30, verbose=2)

### More Complex Models

In [ ]:
model = Sequential([
    layers.Dense(256, input_dim=100, activation="relu", name="input"),
    layers.Dense(128, activation="relu", name="layer1"),
    layers.Dense(64, activation="relu", name="layer2"),
    layers.Dense(5, activation="softmax", name="output")
])
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x=X, y=Y, batch_size=10, epochs=30, verbose=2, validation_split=0.1)